<a href="https://colab.research.google.com/github/anshupandey/ms-generativeai-apr2025/blob/main/code8_parallel_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Parallel Function Calls and Multiple Function Responses

## Overview

Funciton Calling lets you create a description of a function in your code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with.

In this tutorial, you'll learn how to work with parallel function calling including:
    
- Handling parallel function calls for repeated functions
- Working with parallel function calls across multiple functions
- Extracting multiple function calls from a Gemini response
- Calling multiple functions and returning them to Gemini

### What is parallel function calling?

In previous versions of LLMs, it would return two or more chained function calls if the model determined that more than one function call was needed before returning a natural language summary. Here, a chained function call means that you get the first function call response, return the API data to Gemini, get a second function call response, return the API data to Gemini, and so on.

In recent versions of specific Language models (from May 2024 and on), most language models have the ability to return two or more function calls in parallel (i.e., two or more function call responses within the first function call response object). Parallel function calling allows you to fan out and parallelize your API calls or other actions that you perform in your application code, so you don't have to work through each function call response and return one-by-one!


<img src="https://storage.googleapis.com/github-repo/generative-ai/gemini/function-calling/parallel-function-calling-in-gemini.png">

In [1]:
!pip install wikipedia --quiet

In [2]:
from openai import OpenAI
client = OpenAI()

### Define helper function

In [3]:
def extract_tool_calls(response: dict):
    tool_calls = []
    tool_call = response.choices[0].message.tool_calls
    if tool_call:
        for tool in tool_call:
            function_name = tool.function.name
            function_args = eval(tool.function.arguments)  # Safely convert string to dictionary
            function_call_dict = {function_name: function_args}
            function_call_dict['id'] = tool.id
            tool_calls.append(function_call_dict)

    return tool_calls

## Example: Parallel function calls on the same function

A great use case for parallel function calling is when you have a function that only accepts one parameter per API call and you need to make repeated calls to that function.

With Parallel Function Calling, rather than having to send N number of API requests to LLM for N number function calls, instead you can send a single API request to LLM, receive N number of Function Call Responses within a single response, make N number of external API calls in your code, then return all of the API responses to LLM in bulk. And you can do all of this without any extra configuration in your function declarations, tools, or requests to LLM.

In this example, you'll do exactly that and use Parallel Function Calling in LLM to ask about multiple topics on [Wikipedia](https://www.wikipedia.org/). Let's get started!


In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_wikipedia",
            "description": "Search for articles or any other relevant information on Wikipedia'",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    }
]

In [7]:
### Send prompt to OpenAI GPT-4o with function calling
prompt = "Search for articles related to solar panels, renewable energy, and battery storage and provide a summary of your findings"

In [8]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice="auto",
)


In [9]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_xyNY1JAVJbwXdA38XnzzEHbm', function=Function(arguments='{"query": "solar panels"}', name='search_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_oY5JxRgQa0AAf7itFtNiGxDa', function=Function(arguments='{"query": "renewable energy"}', name='search_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_SjRvmCE1MXkoSb19k3sWWdnC', function=Function(arguments='{"query": "battery storage"}', name='search_wikipedia'), type='function')]

In [10]:
response2 = client.chat.completions.create(model='gpt-4o',
                                           messages=[{"role":"user","content":"Define Artificial Intelligence"}],
                                           max_tokens=10,n=2)

print(response2.model_dump_json(indent=2))

{
  "id": "chatcmpl-BIBXJUggOEdBAnADtqGTTF6qq3LFc",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Artificial Intelligence (AI) is a field of computer",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    },
    {
      "finish_reason": "length",
      "index": 1,
      "logprobs": null,
      "message": {
        "content": "Artificial Intelligence (AI) refers to the simulation of",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1743673949,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_898ac29719",
  "usage": {
    "completion_tokens": 20,
    "prompt_tokens": 1

In [ ]:
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-A8MebNy7sWWPtmi9hzBuajutYL0Vn",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_TPg6ABl3zqJQMA7HMKVESV4K",
            "function": {
              "arguments": "{\"query\": \"solar panels\"}",
              "name": "search_wikipedia"
            },
            "type": "function"
          },
          {
            "id": "call_cGfCcd1EPr1vEanUESUn5QN7",
            "function": {
              "arguments": "{\"query\": \"renewable energy\"}",
              "name": "search_wikipedia"
            },
            "type": "function"
          },
          {
            "id": "call_ELni6fchwPvjRi31GZEDadum",
            "function": {
              "arguments": "{\"query\": \"battery storage\"}",
              "name": "search_wikipedia"


In [11]:
# Extract function calls from the response
tool_calls = extract_tool_calls(response)
print(tool_calls)

[{'search_wikipedia': {'query': 'solar panels'}, 'id': 'call_xyNY1JAVJbwXdA38XnzzEHbm'}, {'search_wikipedia': {'query': 'renewable energy'}, 'id': 'call_oY5JxRgQa0AAf7itFtNiGxDa'}, {'search_wikipedia': {'query': 'battery storage'}, 'id': 'call_SjRvmCE1MXkoSb19k3sWWdnC'}]


In [12]:
### Make external API calls
import wikipedia
api_response = []

# Loop over multiple function calls
for tool in tool_calls:
    print(tool)

    # Make external API call
    result = wikipedia.summary(tool["search_wikipedia"]["query"])

    # Collect all API responses
    api_response.append(result)

{'search_wikipedia': {'query': 'solar panels'}, 'id': 'call_xyNY1JAVJbwXdA38XnzzEHbm'}
{'search_wikipedia': {'query': 'renewable energy'}, 'id': 'call_oY5JxRgQa0AAf7itFtNiGxDa'}
{'search_wikipedia': {'query': 'battery storage'}, 'id': 'call_SjRvmCE1MXkoSb19k3sWWdnC'}


In [13]:
for res in api_response:
    print(res)

A solar panel is a device that converts sunlight into electricity by using photovoltaic (PV) cells. PV cells are made of materials that produce excited electrons when exposed to light. These electrons flow through a circuit and produce direct current (DC) electricity, which can be used to power various devices or be stored in batteries. Solar panels are also known as solar cell panels, solar electric panels, or PV modules.
Solar panels are usually arranged in groups called arrays or systems. A photovoltaic system consists of one or more solar panels, an inverter that converts DC electricity to alternating current (AC) electricity, and sometimes other components such as controllers, meters, and trackers. Most panels are in solar farms or rooftop solar panels which  supply the electricity grid.
Some advantages of solar panels are that they use a renewable and clean source of energy, reduce greenhouse gas emissions, and lower electricity bills. Some disadvantages are that they depend on t

In [14]:

### Generate a natural language summary
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},]

messages.append(response.choices[0].message)
for tool,res in zip(tool_calls,api_response):
  messages.append({"tool_call_id": tool['id'],
                   "role": "tool",
                   "name": 'search_wikipedia',
                   "content": res,})


In [15]:
# Now we return all API responses back to GPT-4 to summarize

new_response = client.chat.completions.create(
    model="gpt-4o",
    messages = messages)

In [16]:

# Display the final summary
print(new_response.choices[0].message.content)

Here is a summary of key information related to solar panels, renewable energy, and battery storage:

1. **Solar Panels:**
   - Solar panels convert sunlight into electricity using photovoltaic (PV) cells, which produce direct current (DC) electricity. These panels are often part of a system that includes an inverter to convert DC into alternating current (AC) usable for household and commercial applications.
   - Advantages include the use of a renewable and clean energy source, reduction of greenhouse gas emissions, and potential cost savings on electricity. However, they can be costly upfront and depend on sunlight availability.
   - Solar panel systems are used in various settings, including roofs and solar farms, and can also be combined with battery storage to enhance their utility.

2. **Renewable Energy:**
   - Renewable energy comes from natural resources like sunlight, wind, rain, tides, waves, and geothermal heat, which are replenished on a human timescale.
   - Major source

## Example: Parallel function calls across multiple functions

Another good fit for parallel function calling is when you have multiple, independent functions that you want to be able to call in parallel, which reduces the number of consecutive OpenAI API calls that you need to make and (ideally) reduces the overall response time to the end user who is waiting for a natural language response.

In this example, you'll use Parallel Function Calling in LLM to ask about multiple aspects of topics and articles on [Wikipedia](https://www.wikipedia.org/).


In [17]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_wikipedia",
            "description": "Search for articles or any other relevant information on Wikipedia",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "suggest_wikipedia",
            "description": "Get suggested titles from Wikipedia based on the given query term",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for suggested titles on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "summarize_wikipedia",
            "description": "Retrieve brief summaries of Wikipedia articles related to a given topic",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "Topic to search for article summaries on Wikipedia"
                    }
                },
                "required": ["topic"],
                "additionalProperties": False
            }
        }
    }
]


In [18]:
prompt = "Show the search results, variations, and article summaries about Wikipedia articles related to the solar system"

In [19]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice='auto'
)


In [20]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_aI89TM0JQu7KxNgEt9NZeLT8', function=Function(arguments='{"query": "solar system"}', name='search_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_hDIMDPJPeiFmdjficYlrBq2B', function=Function(arguments='{"query": "solar system"}', name='suggest_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_bn6mlc271ASLsW4N6ZO7nABT', function=Function(arguments='{"topic": "solar system"}', name='summarize_wikipedia'), type='function')]

### Extract function names and parameters

Use the helper function that we created earlier to extract the function names and function parameters for each Function Call that LLM responded with:

In [21]:
# Extract function calls from the response
tool_calls = extract_tool_calls(response)
print(tool_calls)

[{'search_wikipedia': {'query': 'solar system'}, 'id': 'call_aI89TM0JQu7KxNgEt9NZeLT8'}, {'suggest_wikipedia': {'query': 'solar system'}, 'id': 'call_hDIMDPJPeiFmdjficYlrBq2B'}, {'summarize_wikipedia': {'topic': 'solar system'}, 'id': 'call_bn6mlc271ASLsW4N6ZO7nABT'}]


### Make external API calls

Next, you'll loop through the Function Calls and use the `wikipedia` Python package to make APIs calls and gather information from Wikipedia:

In [22]:
api_response = {}

# Loop over multiple function calls
for tool in tool_calls:
    # Extract the function name
    print(tool)
    function_name = list(tool.keys())[0]

    # Determine which external API call to make
    if function_name == "search_wikipedia":
        result = wikipedia.search(tool["search_wikipedia"]["query"])
    if function_name == "suggest_wikipedia":
        result = wikipedia.suggest(tool["suggest_wikipedia"]["query"])
    if function_name == "summarize_wikipedia":
        result = wikipedia.summary(
            tool["summarize_wikipedia"]["topic"], auto_suggest=False
        )

    # Collect all API responses
    api_response[function_name] = result

{'search_wikipedia': {'query': 'solar system'}, 'id': 'call_aI89TM0JQu7KxNgEt9NZeLT8'}
{'suggest_wikipedia': {'query': 'solar system'}, 'id': 'call_hDIMDPJPeiFmdjficYlrBq2B'}
{'summarize_wikipedia': {'topic': 'solar system'}, 'id': 'call_bn6mlc271ASLsW4N6ZO7nABT'}


In [23]:
api_response

{'search_wikipedia': ['Solar System',
  'Formation and evolution of the Solar System',
  'List of Solar System objects by size',
  'Solar System belts',
  'Solar System (TV series)',
  'List of Solar System objects',
  'Photovoltaic system',
  'Sweden Solar System',
  'Solar System model',
  'List of natural satellites'],
 'suggest_wikipedia': 'soler system',
 'summarize_wikipedia': "The Solar System is the gravitationally bound system of the Sun and the objects that orbit it. It formed about 4.6 billion years ago when a dense region of a molecular cloud collapsed, forming the Sun and a protoplanetary disc. The Sun is a typical star that maintains a balanced equilibrium by the fusion of hydrogen into helium at its core, releasing this energy from its outer photosphere. Astronomers classify it as a G-type main-sequence star.\nThe largest objects that orbit the Sun are the eight planets. In order from the Sun, they are four terrestrial planets (Mercury, Venus, Earth and Mars); two gas gi

### Get a natural language summary

Now you can return all of the API responses to LLM so that it can generate a natural language summary:

In [24]:

### Generate a natural language summary
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},]

messages.append(response.choices[0].message)
for tool,res in zip(tool_calls,api_response):
  function_name = list(tool.keys())[0]
  messages.append({"tool_call_id": tool['id'],
                   "role": "tool",
                   "name": function_name,
                   "content": res,})


In [25]:
# Now we return all API responses back to GPT-4 to summarize

new_response = client.chat.completions.create(
    model="gpt-4o",
    messages = messages)

In [26]:
# Display the final summary
print(new_response.choices[0].message.content)

Here is the information I've gathered about the Wikipedia articles related to the solar system:

1. **Search Results**:
   - **Solar System**: Provides a general overview of the Solar System, including its formation and components like planets, moons, comets, asteroids, and the Sun.
   - **Planets of the Solar System**: Details about each planet within the Solar System, including their characteristics and order from the Sun.
   - **Moons of the Solar System**: Discusses the natural satellites that orbit planets, including those of major planets like Jupiter, Saturn, and Earth’s Moon.
   - **Asteroids and Comets**: Articles provide information on smaller bodies within the Solar System, their composition, and their orbits.
   - **Formation and Evolution of the Solar System**: Explores scientific theories on how the Solar System was created and has changed over time.

2. **Variations**:
   - **List of Planets and Dwarf Planets**: Details the recognized planets and dwarf planets, including

# Sequential Multi function call

In [27]:
import requests
import json

In [28]:
# Define the OpenWeatherMap API key
OWM_API_KEY = "29af1cea50a401d8e624eea4660b3f59"

def get_current_weather(location, unit="kelvin"):
    """
    Fetches the current weather information for a given location.

    Parameters:
    - location: str, the name of the location (e.g., "Paris").
    - unit: str, the unit of temperature (default is "kelvin").

    Returns:
    - str: JSON formatted string containing weather information.
    """
    # Construct the API request URL
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={OWM_API_KEY}"

    try:
      # Send the API request
      response = requests.get(url)
    except:
      return "Error occurred because location does not exist"

    # Parse the temperature and weather forecast from the response
    temp = response.json()['main']['temp']
    forecast = [response.json()['weather'][0]['main'], response.json()['weather'][0]['description']]

    # Create a dictionary with the weather information
    weather_info = {
        "location": location,
        "temperature": temp,
        "unit": 'Kelvin',
        "forecast": forecast
    }

    # Return the weather information as a JSON string
    return json.dumps(weather_info)

In [29]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_wikipedia",
            "description": "Search for articles or any other relevant information on Wikipedia'",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    },
    {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit","kelvin"]},
                    },
                    "required": ["location"],
                },
            },
        },
]

In [30]:
prompt = "Get the name of capital city of Tripura using wikipedia and then tell me the weather information for the same using wikipedia results. "

In [31]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice='auto'
)


In [32]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_ISdsXcYX89A7GYcPR6HKQvGL', function=Function(arguments='{"query":"capital of Tripura"}', name='search_wikipedia'), type='function')]

### Extract function names and parameters

Use the helper function that we created earlier to extract the function names and function parameters for each Function Call that LLM responded with:

In [33]:
# Extract function calls from the response
tool_calls = extract_tool_calls(response)
print(tool_calls)

[{'search_wikipedia': {'query': 'capital of Tripura'}, 'id': 'call_ISdsXcYX89A7GYcPR6HKQvGL'}]


In [34]:
### Make external API calls
import wikipedia
api_response = []

# Loop over multiple function calls
for tool in tool_calls:
    print(tool)

    # Make external API call
    result = wikipedia.summary(tool["search_wikipedia"]["query"])

    # Collect all API responses
    api_response.append(result)

{'search_wikipedia': {'query': 'capital of Tripura'}, 'id': 'call_ISdsXcYX89A7GYcPR6HKQvGL'}


In [35]:
for res in api_response:
    print(res)

Tripura () is a state in northeastern India. The third-smallest state in the country, it covers 10,491 km2 (4,051 sq mi); and the seventh-least populous state with a population of 3.67 million. It is bordered by Assam and Mizoram to the east and by Bangladesh to the north, south and west. Tripura is divided into 8 districts and 23 sub-divisions, where Agartala is the capital and the largest city in the state. Tripura has 19 different tribal communities with a majority Bengali population. Bengali, English and Kokborok are the state's official languages.
The area of modern Tripura — ruled for several centuries by the Manikya Dynasty — was part of the Tripuri Kingdom (also known as Hill Tippera). It became a princely state under the British Raj during its tenure, and acceded to independent India in 1947. It merged with India in 1949 and was designated as a 'Part C State' (union territory). It became a full-fledged state of India in 1972.
Tripura lies in a geographically isolated location 

In [36]:
### Generate a natural language summary
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},]

messages.append(response.choices[0].message)
for tool,res in zip(tool_calls,api_response):
  messages.append({"tool_call_id": tool['id'],
                   "role": "tool",
                   "name": 'search_wikipedia',
                   "content": res,})


In [37]:
# Now we return all API responses back to GPT-4 to summarize

new_response = client.chat.completions.create(
    model="gpt-4o",
    messages = messages,
    tools=tools,
    tool_choice='auto')

In [38]:
# Display the final summary
print(new_response.model_dump_json(indent=2))

{
  "id": "chatcmpl-BIBoPkcxQirJ66Pe8YmZgauoLXY81",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_JwdVGeeIhqa9cxYBV98kCHYM",
            "function": {
              "arguments": "{\"location\":\"Agartala, Tripura\",\"unit\":\"celsius\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1743675009,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_898ac29719",
  "usage": {
    "completion_tokens": 26,
    "prompt_tokens": 692,
    "total_tokens": 718,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio

In [ ]:
for i in range(10):
  response = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice='auto')

  tool_call_flag = response.choices[0].message.tool_calls

  if tool_call:
    response = response.choices[0].message
    tool_calls = response.tool_calls
    for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(**function_args)
                print(function_response)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )

  else:
    return response.choices[0].message.content



In [ ]:
tools = [
    {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit","kelvin"]},
                    },
                    "required": ["location"],
                },
            },
        },
]

In [ ]:
prompt = "what is 2+2? and tell me the weather of bangalore. Answer questions together at the end."

In [ ]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice='auto'
)


In [ ]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_lfTIZcEXKhJRzjIF5srNpGKH', function=Function(arguments='{"location": "Bangalore"}', name='get_current_weather'), type='function')]

In [ ]:
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-A8NT540tyItXaWXNw9dALXn1cAoXU",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_lfTIZcEXKhJRzjIF5srNpGKH",
            "function": {
              "arguments": "{\"location\": \"Bangalore\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      },
      "content_filter_results": {}
    }
  ],
  "created": 1726560199,
  "model": "gpt-4o-2024-05-13",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": "fp_80a1bad4c7",
  "usage": {
    "completion_tokens": 32,
    "prompt_tokens": 106,
    "total_tokens": 138,
    "completion_tokens_details": null
  },
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results"

In [ ]:
### Generate a natural language summary
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},]

messages.append(response.choices[0].message)
for tool,res in zip(tool_calls,api_response):
  messages.append({"tool_call_id":"call_lfTIZcEXKhJRzjIF5srNpGKH" ,
                   "role": "tool",
                   "name": 'get_current_weather',
                   "content": '{"temperature":"340","unit":"kelvin"}',})


In [ ]:
# Now we return all API responses back to GPT-4 to summarize

new_response = client.chat.completions.create(
    model="gpt-4o",
    messages = messages,
    tools=tools,
    tool_choice='auto')

In [ ]:
messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': 'what is 2+2? and tell me the weather of bangalore. Answer questions together at the end.'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lfTIZcEXKhJRzjIF5srNpGKH', function=Function(arguments='{"location": "Bangalore"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_lfTIZcEXKhJRzjIF5srNpGKH',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"temperature":"340","unit":"kelvin"}'}]

In [ ]:
# Display the final summary
print(new_response.model_dump_json(indent=2))

{
  "id": "chatcmpl-A8NTbq5K9gylL2fMVglMVefn6K2aj",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "1. The answer to 2+2 is 4.\n2. The current weather in Bangalore is 340 K (Kelvin).",
        "refusal": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1726560231,
  "model": "gpt-4o-2024-05-13",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": "fp_80a1bad4c7",
  "usage": {
    "completion_tokens": 29,
   